# Dataset preparation

#### Imports

In [44]:
#sql
import mysql.connector
from mysql.connector import Error
import pymysql

#dataframe
import pandas as pd
import numpy as np

#visualization
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

Features to find: 
Punk number,Skin, Type, Rarest rank, Total number of attributes, Last sold price, Number of sold transactions, Total in sold transactions, Number of transactions, Highest sold price, Difference between two last sold transactions, Current hold time.

In [45]:
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='mysql',
                             db='mydb')

In [46]:
cursor = connection.cursor()
cursor.execute("USE mydb")

0

#### Queries to get all features

In [47]:
try:
    query = """SELECT id, skin_id, type_id, rarest_rank FROM punk;"""
    df1 = pd.read_sql(query,connection)
except Exception as e:
    cursor.close()
    print(str(e))

In [48]:
df1.rename(columns={'id': 'punk_id'}, inplace=True)

In [49]:
df1.head(2)

,punk_id,skin_id,type_id,rarest_rank
0,0,6,3,2118
1,1,4,4,4337


In [50]:
try:
    query = """SELECT punk_id, attribute_id FROM punk_attribute_relation;"""
    df2 = pd.read_sql(query,connection)
except Exception as e:
    cursor.close()
    print(str(e))

In [51]:
df2 = df2.groupby(['punk_id']).size()
df2 = df2.to_frame()
df2.columns = ['total_att']
df2['punk_id'] = df2.index
df2 = df2.reset_index(drop=True)
df2.head(2)

,total_att,punk_id
0,3,0
1,2,1


In [52]:
len(df2)

9992

In [53]:
new_row = {'total_att':0, 'punk_id':281}
df2 = df2.append(new_row, ignore_index=True)
new_row = {'total_att':0, 'punk_id':510}
df2 = df2.append(new_row, ignore_index=True)
new_row = {'total_att':0, 'punk_id':641}
df2 = df2.append(new_row, ignore_index=True)
new_row = {'total_att':0, 'punk_id':741}
df2 = df2.append(new_row, ignore_index=True)
new_row = {'total_att':0, 'punk_id':1050}
df2 = df2.append(new_row, ignore_index=True)
new_row = {'total_att':0, 'punk_id':2204}
df2 = df2.append(new_row, ignore_index=True)
new_row = {'total_att':0, 'punk_id':3307}
df2 = df2.append(new_row, ignore_index=True)
new_row = {'total_att':0, 'punk_id':6487}
df2 = df2.append(new_row, ignore_index=True)
len(df2)

10000

In [54]:
df2 = df2.sort_values(by=['punk_id'])
df2 = df2.reset_index(drop=True)
df2.head(3)

,total_att,punk_id
0,3,0
1,2,1
2,1,2


In [55]:
try:
    query = """SELECT date, amount, punk_id 
    FROM transaction
    WHERE type = 'Sold';"""
    df3 = pd.read_sql(query,connection)
except Exception as e:
    cursor.close()
    print(str(e))

In [56]:
#keep only first item for each punk
df3 = df3.drop_duplicates(subset=['punk_id'], keep='first')
df3.rename(columns={'amount': 'last_price'}, inplace=True)
df3.drop(['date'], axis=1, inplace=True)
df3.head(5)

,last_price,punk_id
0,25.00,0
3,60.00,1
6,2.50,14
9,0.51,33
10,129.99,53


In [57]:
len(df3)

6500

In [58]:
try:
    query = """SELECT date, amount, punk_id 
    FROM transaction
    WHERE type = 'Sold';"""
    df4 = pd.read_sql(query,connection)
except Exception as e:
    cursor.close()
    print(str(e))

In [59]:
df4 = df4.groupby(['punk_id']).count()
df4['punk_id'] = df4.index
df4 = df4.reset_index(drop=True)
df4.drop(['date'], axis=1, inplace=True)
df4.rename(columns={'amount': 'num_sold_trans'}, inplace=True)
df4.head(5)

,num_sold_trans,punk_id
0,3,0
1,3,1
2,3,14
3,1,33
4,3,53


In [60]:
len(df4)

6500

In [61]:
try:
    query = """SELECT date, amount, punk_id 
    FROM transaction
    WHERE type = 'Sold';"""
    df5 = pd.read_sql(query,connection)
except Exception as e:
    cursor.close()
    print(str(e))

In [62]:
df5 = df5.groupby(['punk_id']).sum()
df5['punk_id'] = df5.index
df5 = df5.reset_index(drop=True)
df5.rename(columns={'amount': 'total_sold_trans'}, inplace=True)
df5.head(5)

,total_sold_trans,punk_id
0,27.58,0
1,91.42,1
2,5.25,14
3,0.51,33
4,171.99,53


In [63]:
len(df5)

6500

In [64]:
try:
    query = """SELECT date, amount, punk_id 
    FROM transaction;"""
    df6 = pd.read_sql(query,connection)
except Exception as e:
    cursor.close()
    print(str(e))

In [65]:
df6 = df6.groupby(['punk_id']).count()
df6['punk_id'] = df6.index
df6.drop(['date'], axis=1, inplace=True)
df6 = df6.reset_index(drop=True)
df6.rename(columns={'amount': 'num_trans'}, inplace=True)
df6.head(5)

,num_trans,punk_id
0,40,0
1,63,1
2,30,2
3,17,3
4,12,4


In [66]:
try:
    query = """SELECT date, amount, punk_id 
    FROM transaction
    WHERE type = 'Sold';"""
    df7 = pd.read_sql(query,connection)
except Exception as e:
    cursor.close()
    print(str(e))

In [67]:
df7 = df7.sort_values(["punk_id","amount"], ascending = (True, False))
df7.head(3)

,date,amount,punk_id
0,2018-11-30,25.00,0
1,2017-07-07,1.60,0
2,2017-06-23,0.98,0


In [68]:
df7 = df7.drop_duplicates(subset=['punk_id'], keep='first')
df7.rename(columns={'amount': 'highest_price'}, inplace=True)
df7.drop(['date'], axis=1, inplace=True)
df7.head(3)

,highest_price,punk_id
0,25.0,0
3,60.0,1
6,2.5,14


In [69]:
try:
    query = """SELECT date, amount, punk_id 
    FROM transaction
    WHERE type = 'Sold';"""
    df8 = pd.read_sql(query,connection)
except Exception as e:
    cursor.close()
    print(str(e))

In [70]:
df8.rename(columns={'amount': 'last_price'}, inplace=True)
df8.drop(['date'], axis=1, inplace=True)
df8.head(3)

,last_price,punk_id
0,25.00,0
1,1.60,0
2,0.98,0


In [71]:
#last price
df8a = df8.groupby('punk_id').head(1)
df8a = df8a.reset_index(drop=True)
#previous price
df8b = df8.groupby('punk_id').head(2)
df8b = df8b.drop_duplicates(subset=['punk_id'], keep='last')
df8b = df8b.reset_index(drop=True)
df8b.rename(columns={'last_price': 'previous_price'}, inplace=True)

In [72]:
df8 = df8a.copy()
df8['last_prices_diff'] = (df8a['last_price']) - (df8b['previous_price'])
df8.drop(['last_price'], axis=1, inplace=True)
df8.head(10)

,punk_id,last_prices_diff
0,0,23.40
1,1,29.00
2,14,0.65
3,33,0.00
4,53,107.99
5,89,-0.01
6,116,1.20
7,118,0.20
8,124,-2.90
9,138,15.99


In [73]:
try:
    query = """SELECT date, amount, punk_id 
    FROM transaction
    WHERE type = 'Sold';"""
    df9 = pd.read_sql(query,connection)
except Exception as e:
    cursor.close()
    print(str(e))

In [74]:
#last price
df9 = df9.groupby('punk_id').head(1)
df9 = df9.reset_index(drop=True)

In [75]:
df9.rename(columns={'amount': 'last_price','date': 'last_date'}, inplace=True)
df9.head(3)

,last_date,last_price,punk_id
0,2018-11-30,25.0,0
1,2020-11-30,60.0,1
2,2018-12-07,2.5,14


In [76]:
import datetime
x = datetime.date(2022,1,10)
y = datetime.date(2017,6,23)
print('Total time of CryptoPunks project', x-y)

Total time of CryptoPunks project 1662 days, 0:00:00


In [77]:
df9['date_diff'] = x - (df9['last_date'])
df9.drop(['last_price','last_date'], axis=1, inplace=True)
df9.head(3)

,punk_id,date_diff
0,0,1137 days
1,1,406 days
2,14,1130 days


In [78]:
df9['date_diff'] = df9['date_diff'].dt.days
df9.head(3)

,punk_id,date_diff
0,0,1137
1,1,406
2,14,1130


#### Combine to final dataframe

In [37]:
#punks and number of attributes
df_final = pd.merge(df1, df2, on=["punk_id"])

#last soldprice
df_final = df_final.merge(df3, how='left')
df_final['last_price'] = df_final['last_price'].fillna(0)

#number of sold transactions
df_final = df_final.merge(df4, how='left')
df_final['num_sold_trans'] = df_final['num_sold_trans'].fillna(0)

#total in sold transactions
df_final = df_final.merge(df5, how='left')
df_final['total_sold_trans'] = df_final['total_sold_trans'].fillna(0)

#number of transactions
df_final = df_final.merge(df6, how='left')
df_final['num_trans'] = df_final['num_trans'].fillna(0)

#highest price
df_final = df_final.merge(df7, how='left')
df_final['highest_price'] = df_final['highest_price'].fillna(0)

#difference in last prices
df_final = df_final.merge(df8, how='left')
df_final['last_prices_diff'] = df_final['last_prices_diff'].fillna(0)

#current hold time
df_final = df_final.merge(df9, how='left')
df_final['date_diff'] = df_final['date_diff'].fillna(1662)

df_final.head(3)

,punk_id,skin_id,type_id,rarest_rank,total_att,last_price,num_sold_trans,total_sold_trans,num_trans,highest_price,last_prices_diff,date_diff
0,0,6,3,2118,3,25.0,3.0,27.58,40,25.0,23.4,1137.0
1,1,4,4,4337,2,60.0,3.0,91.42,63,60.0,29.0,406.0
2,2,5,3,7995,1,0.0,0.0,0.00,30,0.0,0.0,1662.0


In [38]:
df_final['highest_price'].max()

4200.0

In [39]:
df_final = df_final[df_final['last_price'] < 120]
df_final = df_final[df_final['highest_price'] < 120]

In [40]:
print(df_final['last_price'].max(),df_final['highest_price'].max())

119.99 119.99


In [42]:
df = df_final.loc[(df_final['type_id'] == 3) | (df_final['type_id'] == 4)]
df.head(10)

,punk_id,skin_id,type_id,rarest_rank,total_att,last_price,num_sold_trans,total_sold_trans,num_trans,highest_price,last_prices_diff,date_diff
0,0,6,3,2118,3,25.0,3.0,27.58,40,25.0,23.4,1137.0
1,1,4,4,4337,2,60.0,3.0,91.42,63,60.0,29.0,406.0
2,2,5,3,7995,1,0.0,0.0,0.00,30,0.0,0.0,1662.0
3,3,4,4,7834,3,0.0,0.0,0.00,17,0.0,0.0,1662.0
4,4,6,4,7189,4,0.0,0.0,0.00,12,0.0,0.0,1662.0
5,5,6,3,2073,3,0.0,0.0,0.00,16,0.0,0.0,1662.0
6,6,6,4,4174,1,0.0,0.0,0.00,13,0.0,0.0,1662.0
7,7,6,3,937,3,0.0,0.0,0.00,14,0.0,0.0,1662.0
8,8,4,4,6394,2,0.0,0.0,0.00,19,0.0,0.0,1662.0
9,9,6,4,1816,3,0.0,0.0,0.00,31,0.0,0.0,1662.0


In [43]:
df.to_csv('final_dataset.csv')